# Optimus Prime x McKinsey & Company
### Alejandro Campayo, Álvaro Domingo, Victor Sainz, Maria Zyatyugina

This code aims to predict the future amount of sales of a particular products given a speciefic date. To face this challenge, Optimus Prime team trained a Multihead Transformer model with 5 heads. The model uses the time series to find trends within the given dataset. All sales are assumed to have a weekly relation, for example, to predict an event happenning on Monday the algorithm will put attention on 7 previous days with self-attention algorithm.

## Imports

In [19]:
from types import SimpleNamespace
from collections import Counter
import os
import re
import pathlib
import array
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import math
import datetime

## Import datasets

To execute this code, please, specify the path to the files on your machine

In [20]:
geo_params = pd.read_csv("../input/mckinsey/geo_params.csv")
test = pd.read_csv("../input/mckinsey/test.csv")
sku = pd.read_csv("../input/mckinsey/sku.csv")
sales = pd.read_csv("../input/mckinsey/sales.csv")
sales_mat = np.load("../input/sales-matrix/sales_expanded.npy")[:,:,:,1]

## Model definition

In [21]:
params = SimpleNamespace(
    window_size = 7,
    batch_size = 2048,
    epochs = 4,
    geo_dim = 515,
    hidden_dim = 515,
    prod_dim = 60,
    train = True
)
num_heads = 5

In [22]:
# Select device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    print("WARNING: Training without GPU can be very slow!")

In [23]:
n_sku = len(sku)
n_geo = len(geo_params)

# Transforms a given SKU to index
dict_sku = {}
for i_sku in range(n_sku):
    sku_id = sku['SKU'][i_sku]
    dict_sku[sku_id] = i_sku
    
# Transforms a given geo_Cluster to index
dict_geo = {}
for i_geo in range(n_geo):
    geo_id = geo_params['geoCluster'][i_geo]
    dict_geo[geo_id] = i_geo

In [24]:
def get_day(sku_id, date_str):
    
    sk = dict_sku[int(sku_id)]
    try:
        return sales_mat[date_str,sk,:,:].flatten()
    except:
        return np.zeros(params.geo_dim)

In [25]:
def batch_generator(idata, target, batch_size, shuffle=True):
    nsamples = len(idata)
    if shuffle:
        perm = np.random.permutation(nsamples)
    else:
        perm = range(nsamples)

    for i in range(0, nsamples, batch_size):
        batch_idx = perm[i:i+batch_size]
        if target is not None:
            yield idata[batch_idx], target[batch_idx]
        else:
            yield idata[batch_idx], None

In [26]:
def lossfunction (pred, real):
    for i in range (len(pred)):
        for j in range(len(pred[0])):
            if np.isnan(real[i][j].cpu()):
                real[i][j] = pred[i][j]
    loss = torch.mean((real-pred)**2)
    return loss

In [27]:
def train(model, criterion, optimizer, idata, target, batch_size, device, log=False):
    model.train()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    for X, y in batch_generator(idata, target, batch_size, shuffle=True):
        # Get input and target sequences from batch
        X = torch.tensor(X, dtype=torch.long, device=device)
        y = torch.tensor(y, dtype=torch.long, device=device)

        model.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        
        # Training statistics
        total_loss += loss.item()
        niterations += 1
        if niterations == 200 or niterations == 500 or niterations % 1000 == 0:
            print(f'MSE={loss:.2f}')

    print(f'TOTAL MSE={total_loss:.2f}')
    return total_loss

In [28]:
def validate(model, criterion, idata, target, batch_size, device):
    model.eval()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    y_pred = []
    with torch.no_grad():
        for X, y in batch_generator(idata, target, batch_size, shuffle=False):
            # Get input and target sequences from batch
            X = torch.tensor(X, dtype=torch.long, device=device)
            output = model(X)
            if target is not None:
                y = torch.tensor(y, dtype=torch.long, device=device)
                loss = criterion(output, y)
                total_loss += loss.item()
                niterations += 1
            else:
                pred = torch.max(output, 1)[1].detach().to('cpu').numpy()
                y_pred.append(pred)

    if target is not None:
        return total_loss
    else:
        return np.concatenate(y_pred)

In [29]:
def attention(q, k, v, mask=None, dropout=None):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [30]:
class SelfMultiAttention(nn.Module):
    def __init__(self, geo_dim, num_heads, hidden_dim = None, bias=True):
        super().__init__()
        if hidden_dim is None:
            hidden_dim = geo_dim
        assert geo_dim % num_heads == 0, "Geo dimension must be 0 modulo number of heads."
        self.num_heads = num_heads
        self.head_dim = geo_dim // num_heads
        self.qkv_proj = nn.Linear(geo_dim, 3*geo_dim,bias=bias)
        self.k_proj = nn.Linear(geo_dim, geo_dim, bias=bias)
        self.v_proj = nn.Linear(geo_dim, geo_dim, bias=bias)
        self.q_proj = nn.Linear(geo_dim, geo_dim, bias=bias)
        self.out_proj = nn.Linear(geo_dim, hidden_dim, bias=bias)
        self.reset_parameters()

    def reset_parameters(self):
        # Empirically observed the convergence to be much better with the scaled initialization
        nn.init.xavier_uniform_(self.k_proj.weight)
        self.k_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.v_proj.weight)
        self.v_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.q_proj.weight)
        self.q_proj.bias.data.fill_(0)
        nn.init.xavier_uniform_(self.out_proj.weight)
        self.out_proj.bias.data.fill_(0)
        if self.out_proj.bias is not None:
            nn.init.constant_(self.out_proj.bias, 0.)

    def forward(self, x, mask = None):
        seq_length = x.size()[1]
        qkv = self.qkv_proj(x)
        
        try:
            qkv = qkv.reshape(params.batch_size, seq_length, self.num_heads, 3*self.head_dim)
        except:
            batch_size = qkv.size()[0]
            qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
            
        qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
        q, k, v = qkv.chunk(3, dim=-1)
        y, _ = attention(q, k, v, mask = mask) #Attention layer
        y = y.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
        
        try: 
            y = y.reshape(params.batch_size, seq_length , params.geo_dim)
        except:
            batch_size = y.size()[0]
            y = y.reshape(batch_size, seq_length , params.geo_dim)
            
        y = self.out_proj(y)
        return y

In [31]:
class TransformerLayer(nn.Module):
    def __init__(self, d_model, num_heads, dim_feedforward=512, dropout=0.1, activation="relu"):
        super().__init__()
        self.self_attn = SelfMultiAttention(d_model,num_heads)
        
        # Implementation of Feedforward model
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        src2 = self.self_attn(src)
        src = src + self.dropout1(src2)
        src = self.norm1(src)
        src2 = self.linear2(self.dropout(F.relu(self.linear1(src))))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src

In [32]:
class Predictor(nn.Module):
    def __init__(self, geo_dim, prod_dim, num_heads, get_day = get_day, hidden_dim = None, context_dates=params.window_size):
        super().__init__()
        if hidden_dim is None:
            hidden_dim = geo_dim
        self.get_day = get_day
        self.num_heads = num_heads
        self.lin = nn.Linear(hidden_dim + prod_dim, geo_dim, bias=False)
        self.att = TransformerLayer(hidden_dim,num_heads)
        self.position_embedding = nn.Parameter(torch.Tensor(context_dates, geo_dim))
        nn.init.xavier_uniform_(self.position_embedding)

    def forward(self, inp):
        d = []
        z = torch.zeros((inp.shape[0],60)).cuda()
        for i, batch in enumerate(inp):
            b = []
            for sku, date in batch:
                b.append(self.get_day(sku,date))
            d.append(b)
            z[i][dict_sku[int(sku)]] = 1

        d = torch.Tensor(d)
        u = d.cuda() + self.position_embedding
        v = self.att(u) #Transformer layer
        x = v.sum(dim = 1)
        c = torch.cat((x,z),dim = 1)
        y = self.lin(c)
        return y

In [33]:
min_date = sales.groupby(["SKU"]).min()
max_date = sales.groupby(["SKU"]).max()
inp = [[],[]]
out = [[],[]]
for s in sku['SKU']:
    date_min = min_date[min_date.index == s]
    date_max = max_date[max_date.index == s]
    ms = list(date_min['date'])[0].split("-")
    min_s = datetime.datetime(int(ms[0]),int(ms[1]),int(ms[2]))
    
    ms = list(date_max['date'])[0].split("-")
    max_s = datetime.datetime(int(ms[0]),int(ms[1]),int(ms[2]))
    
    diff = (max_s - min_s).days
    d = min_s
    
    #The earliest date found in the dataset is the 2020/01/01 used to calculate the index
    while d + datetime.timedelta(7) < max_s - datetime.timedelta(diff // 10):
        inp[0].append(np.array([(s,(d + datetime.timedelta(i) - datetime.datetime(2020,1,1)).days) for i in range (7)]))
        out[0].append(get_day(s,(d + datetime.timedelta(7) - datetime.datetime(2020,1,1)).days))
        d += datetime.timedelta(1)
        
    while d + datetime.timedelta(7) < max_s:
        inp[1].append(np.array([(s,(d + datetime.timedelta(i) - datetime.datetime(2020,1,1)).days) for i in range (7)]))
        out[1].append(get_day(s,(d + datetime.timedelta(7) - datetime.datetime(2020,1,1)).days))
        d += datetime.timedelta(1)
data = np.array([[inp[0],out[0]], [inp[1],out[1]]])

In [34]:
model = Predictor(params.geo_dim,params.prod_dim,num_heads).to(device)

In [35]:
print(model)
for name, param in model.named_parameters():
    print(f'{name:20} {param.numel()} {list(param.shape)}')
print(f'TOTAL                {sum(p.numel() for p in model.parameters())}')

## Train and Validation
In this section the model is trained with 90% of the dataset and validated with the remaining 10% dataset. The train and validation partition is generated over different products. The data variable contains the dataset partition, the first position contains the train partition whether the second position contains the validation data. 

The performance of the model is measured with the MSE.

In [36]:
optimizer = torch.optim.Adam(model.parameters())
criterion = lossfunction

train_accuracy = []
valid_accuracy = []

for epoch in range(params.epochs):
    loss = train(model, criterion, optimizer, np.array(data[0][0]), np.array(data[0][1]), params.batch_size, device, log=True)
    print(f'| epoch {epoch:03d} | train loss={loss:.2f}')
    loss = validate(model, criterion, np.array(data[1][0]), np.array(data[1][1]), params.batch_size, device)
    print(f'| epoch {epoch:03d} | valid loss={loss:.2f}')


## Test

In order to see the real application of the predictor algorithm the model must be ran over the test data.

In [37]:
test = test.sort_values('date')
test_data = []
test_info = []
for r in test.iterrows():
    test_sku = r[1]['SKU']
    ms = r[1]['date'].split('-')
    test_date = datetime.datetime(int(ms[0]),int(ms[1]),int(ms[2]))
    week = np.array([(test_sku,(test_date - datetime.timedelta(i) - datetime.datetime(2020,1,1)).days) for i in range (7,0,-1)])
    test_data.append(np.array(week))
    test_info.append((test_sku, (test_date - datetime.datetime(2020,1,1)).days, dict_geo[r[1]['geoCluster']]))
test_data = np.array(test_data)

## Filling the test file

The prediction the sales is added to the final array in batches in order to optimize the memory usage.

In [38]:
pred_sales = []
with torch.no_grad():
    for i in range (0,len(test_data),100):
        pred = model(test_data[i : i+100])
        for p, g in zip(pred,test_info[i:i+100]):
            sales_mat[g[1],dict_sku[g[0]],:] = p.cpu()
            pred_sales.append(max(0,float(p[g[2]])))
test['sales'] = pred_sales

In [39]:
test = test.sort_values(['geoCluster', 'SKU'])
test.to_csv("test_test.csv")